In [ ]:
!pip install sagemaker --upgrade
!pip install roboflow pycocotools seaborn
!pip install boto3

### Load and Prep. Dataset

In [ ]:
!unzip dataset_v3.zip -d ./

In [ ]:
!mkdir -p ./data/custom
!mkdir -p ./data/custom/annotations/
!mkdir ./outputs

!mv ./dataset_v3/test.json ./data/custom/annotations/instances_val2017.json
!mv ./dataset_v3/train.json ./data/custom/annotations/instances_train2017.json

!mkdir -p ./data/custom/train2017/
!mv ./dataset_v3/train/images/*.jpg ./data/custom/train2017/

!mkdir -p ./data/custom/val2017/
!mv ./dataset_v3/test/images/*.jpg ./data/custom/val2017/

### Test & Vis. Annotated dataset

In [ ]:
%matplotlib inline
import pycocotools.coco as coco
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [ ]:
dataDir='./data/custom'
dataType='train2017'
annFile='{}/annotations/custom_train.json'.format(dataDir)

In [ ]:
coco=COCO(annFile)

In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())

nms=[cat['name'] for cat in cats]
print('Categories: {}'.format(nms))

nms = set([cat['supercategory'] for cat in cats])
print('Super-categories: {}'.format(nms))

In [ ]:
# load and display image
catIds = coco.getCatIds(catNms=['-'])
imgIds = coco.getImgIds(catIds=catIds)

img_id = imgIds[np.random.randint(0,len(imgIds))]
print('Image n°{}'.format(img_id))

img = coco.loadImgs(img_id)[0]

img_name = '%s/%s/%s'%(dataDir, dataType, img['file_name'])
print('Image name: {}'.format(img_name))

annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds)
anns = coco.loadAnns(annIds)
print(anns)

I = io.imread(img_name)
plt.imshow(I)
coco.showAnns(anns, draw_bbox=True, )

### Loading code base

In [ ]:
!git clone https://github.com/ixd-ai-hub/meter-reader-ai-model.git --branch detr_train
!mv ./meter-reader-ai-model/* .
!rm -rf meter-reader-ai-model